In [40]:
import pandas as pd
import os
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [28]:
df = pd.read_csv(os.path.join("..","Resources","exoplanet_data.csv"))
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
koi = df.dropna()
print(f"Number rows: {len(df)}")
koi.head()

Number rows: 6991


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [29]:
koi = pd.get_dummies(df,drop_first=True)
koi.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_disposition_CONFIRMED,koi_disposition_FALSE POSITIVE
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1,0
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,0,1
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0,1
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,1,0
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714,1,0


In [30]:
# Set features. This will also be used as your x values.
selected_features = ['koi_impact', 'koi_depth', 'koi_prad', 'koi_teq','koi_tce_plnt_num',\
    'koi_steff','koi_slogg','koi_srad']

data = df[selected_features]
feature_names = data.columns
data.head()

,koi_impact,koi_depth,koi_prad,koi_teq,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad
0,0.586,874.8,2.83,443,2,5455,4.467,0.927
1,0.969,10829.0,14.60,638,1,5853,4.544,0.868
2,1.276,8079.2,33.46,1395,1,5805,4.564,0.791
3,0.701,603.3,2.75,1406,1,6031,4.438,1.046
4,0.762,686.0,2.77,1160,2,6046,4.486,0.972


In [31]:
target_names = ['CANDIDATE', 'FALSE POSITIVE']
target = koi.koi_disposition_CONFIRMED.values.reshape(-1, 1)
target

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [33]:
X_train.head()

,koi_impact,koi_depth,koi_prad,koi_teq,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad
6122,0.150,123.1,1.24,1017,1,5737,4.327,1.125
6370,0.291,114.6,0.86,1867,1,5855,4.578,0.797
2879,0.970,641.1,3.21,989,1,6328,4.481,0.963
107,0.300,875.4,2.25,696,1,4768,4.536,0.779
29,0.831,9802.0,12.21,1103,1,5712,4.359,1.082


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [34]:
# Scale your data
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [70]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [36]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.09916952593468897
Testing Data Score: 0.08338086462902905


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [63]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
#model.get_params().keys()
param_grid = {'fit_intercept':np.array([.1, .005, .001]), 
    'normalize':np.array([.1, .005, .001]), 
    'copy_X':np.array([.1, .005, .001]), 
    'n_jobs':np.array([.1, .005, .001])
    }
grid = GridSearchCV(model, param_grid, verbose=3)

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize', 'positive'])

In [65]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV 1/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.1;, score=0.113 total time=   0.0s
[CV 2/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.1;, score=0.080 total time=   0.0s
[CV 3/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.1;, score=0.087 total time=   0.0s
[CV 4/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.1;, score=0.072 total time=   0.0s
[CV 5/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.1;, score=0.116 total time=   0.0s
[CV 1/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.005;, score=0.113 total time=   0.0s
[CV 2/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.005;, score=0.080 total time=   0.0s
[CV 3/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.005;, score=0.087 total time=   0.0s
[CV 4/5] END copy_X=0.1, fit_intercept=0.1, n_jobs=0.1, normalize=0.005;, score=0.072 total time=   0.0s
[CV

GridSearchCV(estimator=LinearRegression(),
             param_grid={'copy_X': array([0.1  , 0.005, 0.001]),
                         'fit_intercept': array([0.1  , 0.005, 0.001]),
                         'n_jobs': array([0.1  , 0.005, 0.001]),
                         'normalize': array([0.1  , 0.005, 0.001])},
             verbose=3)

In [66]:
print(grid.best_params_)
print(grid.best_score_)

{'copy_X': 0.1, 'fit_intercept': 0.1, 'n_jobs': 0.1, 'normalize': 0.1}
0.09369340921555695


# Save the Model

In [68]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'LinearRegression.sav'
joblib.dump(model, filename)

['LinearRegression.sav']